**Author**: J W Debelius<br>
**Date**: 24 March 2015<br>
**virtualenv**: Playground

In [1]:
%%javascript
IPython.load_extensions('calico-spell-check', 'calico-document-tools')

<IPython.core.display.Javascript object>

On GitHub, Daniel ([@wasade](https://github.com/wasade)) asked
<blockquote>
Can you please use the cProfile? This is too specific while masking
interaction with operators making it hard to interpret. It would be useful
to make sure the test method is included as well. You can use cProfile in
IPython by just doing %prun or %%prun.
</blockquote>

So, I'm trying to figure out cProfile. I'm refering to [Profiling your code easily with cProfile and IPython](http://nbviewer.ipython.org/github/dboyliao/cookbook-code/blob/master/notebooks/chapter04_optimization/02_profile.ipynb) by [@dboyliao](https://github.com/dboyliao)

Much of what I'm going to do is coming from what I did on Feburary 27.

In [2]:
import os
import numpy as np
import pandas as pd
import biom
import skbio
import scipy
import matplotlib.pyplot as plt

%matplotlib inline

We're going to read in the American Gut table, and we'll test alpha diversity with a kruskal-wallis test.

In [3]:
base_dir = os.path.abspath('../..')
data_dir = os.path.join(base_dir, 'data/ag_fecal_one_sample_sub')

otu_ = biom.load_table(os.path.join(data_dir, 'AGP_100nt_even10k_fecal.biom'))
map_ = pd.read_csv(os.path.join(data_dir, 'AGP_100nt_even10k_fecal.txt'),
                   sep='\t',
                   index_col=None)
map_.index = map_['#SampleID']

In [4]:
def test_alpha(x):
    vals = [map_.loc[i, 'PD_whole_tree_mean'].values for i in x]
    return scipy.stats.kruskal(*vals)[1]

I'm going to use the release version of scikit-bio, becuase I find maintaing the development versions difficult. 

In [5]:
daily = map_.loc[map_.ALCOHOL_FREQUENCY == 'Daily'].index.values
never = map_.loc[map_.ALCOHOL_FREQUENCY == 'Never'].index.values

In [6]:
from skbio.stats.power import subsample_paired_power

In [7]:
%%prun -s cumulative -q -l 10 -T prun0
#We're going to profile the cell, sorting relative to cumulative time. 
# We'll limit to the first 10 lines and save them to a file, "prun0"
pwr, cnts = subsample_paired_power(test_alpha,
                                   map_,
                                   'ALCOHOL_FREQUENCY',
                                   ['AGE_CAT', 'TYPES_OF_PLANTS', 
                                    'COLLECTION_SEASON', 'SLEEP_DURATION'],
                                   ['Never', 'Daily'],
                                   num_iter=500,
                                   counts_interval=2,
                                   min_observations=15,
                                   max_counts=16)

 
*** Profile printout saved to text file u'prun0'. 


In [8]:
print(open('prun0', 'r').read())

         35954115 function calls (35595655 primitive calls) in 46.787 seconds

   Ordered by: cumulative time
   List reduced from 332 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   46.787   46.787 <string>:4(<module>)
        1    0.001    0.001   46.787   46.787 power.py:328(subsample_paired_power)
       10    0.002    0.000   45.913    4.591 power.py:916(_calculate_power_curve)
       70    0.323    0.005   45.909    0.656 power.py:829(_compare_distributions)
    35000    0.549    0.000   44.418    0.001 <ipython-input-4-640c322d58e3>:1(test_alpha)
140820/70820    0.146    0.000   38.698    0.001 indexing.py:1198(__getitem__)
    70000    0.053    0.000   38.032    0.001 indexing.py:697(_getitem_tuple)
    70000    0.692    0.000   37.979    0.001 indexing.py:798(_getitem_lowerdim)
   140820    0.566    0.000   36.379    0.000 indexing.py:1311(_getitem_axis)
    70820    0.817    0.000   32.399   

In [9]:
%%prun -s time -q -l 10 -T prun1
#We're going to profile the cell, sorting relative to total time. 
# We'll limit to the first 10 lines and save them to a file, "prun1"
pwr, cnts = subsample_paired_power(test_alpha,
                                   map_,
                                   'ALCOHOL_FREQUENCY',
                                   ['AGE_CAT', 'TYPES_OF_PLANTS', 
                                    'COLLECTION_SEASON', 'SLEEP_DURATION'],
                                   ['Never', 'Daily'],
                                   num_iter=500,
                                   counts_interval=2,
                                   min_observations=15,
                                   max_counts=16)

 
*** Profile printout saved to text file u'prun1'. 


In [10]:
print(open('prun1', 'r').read())

         35953849 function calls (35595389 primitive calls) in 45.959 seconds

   Ordered by: internal time
   List reduced from 329 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    70710    2.751    0.000    3.534    0.000 index.py:1578(isin)
  7235790    2.131    0.000    2.752    0.000 {isinstance}
212450/212370    2.115    0.000    7.830    0.000 index.py:125(__new__)
   991230    1.905    0.000    1.905    0.000 {numpy.core.multiarray.array}
   283140    1.561    0.000    3.449    0.000 common.py:2279(_asarray_tuplesafe)
   212400    1.548    0.000    1.548    0.000 {pandas.lib.infer_dtype}
   216160    1.296    0.000    1.296    0.000 {method 'reduce' of 'numpy.ufunc' objects}
   140820    1.239    0.000   10.204    0.000 indexing.py:1240(_has_valid_type)
2046180/1976180    0.949    0.000    0.981    0.000 {getattr}
    70820    0.804    0.000   31.789    0.000 indexing.py:930(_getitem_iterable)


*Note*: The [Python reference](https://docs.python.org/2/library/profile.html) for cProfile is also quite helpful!

------
Daniel asked that I use a simplier test function, since he thinks this is masking bottlenecks in the code. So, I'm going to use a normal distribution, and test on `subsample_power` instead of subsample_paired_power.

In [11]:
# Draws two random samples
sample1 = np.random.randn(50)*1.5
sample2 = np.random.randn(50)*1.5 + 2

In [12]:
# Defines the test
def test(x):
    return scipy.stats.ttest_ind(x[0], x[1])[1]

In [13]:
test([sample1, sample2])

2.5651809734110741e-09

In [14]:
from skbio.stats.power import subsample_power

In [16]:
%%prun -s cumulative -q -l 10 -T prun2
#We're going to profile the cell, sorting relative to cumulative time. 
# We'll limit to the first 10 lines and save them to a file, "prun0"
pwr, cnts = subsample_power(test,
                            [sample1, sample2],
                            num_iter=500,
                            counts_interval=5,
                            min_observations=15,
                            max_counts=31)

 
*** Profile printout saved to text file u'prun2'. 


In [17]:
print(open('prun2', 'r').read())

         4353065 function calls in 9.478 seconds

   Ordered by: cumulative time
   List reduced from 72 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    9.478    9.478 <string>:4(<module>)
        1    0.000    0.000    9.478    9.478 power.py:152(subsample_power)
       10    0.001    0.000    9.478    0.948 power.py:916(_calculate_power_curve)
       60    0.254    0.004    9.474    0.158 power.py:829(_compare_distributions)
    30000    0.041    0.000    7.556    0.000 <ipython-input-12-1563e693a5e4>:2(test)
    30000    0.397    0.000    7.515    0.000 stats.py:3271(ttest_ind)
    30000    0.183    0.000    3.745    0.000 stats.py:3262(_ttest_finish)
    30000    0.841    0.000    3.562    0.000 _distn_infrastructure.py:1701(sf)
    60000    0.089    0.000    2.363    0.000 fromnumeric.py:2830(var)
    60000    1.186    0.000    2.274    0.000 _methods.py:77(_var)


In [18]:
%%prun -s time -q -l 10 -T prun3
#We're going to profile the cell, sorting relative to cumulative time. 
# We'll limit to the first 10 lines and save them to a file, "prun0"
pwr, cnts = subsample_power(test,
                            [sample1, sample2],
                            num_iter=500,
                            counts_interval=5,
                            min_observations=15,
                            max_counts=31)

 
*** Profile printout saved to text file u'prun3'. 


In [19]:
print(open('prun3', 'r').read())

         4353065 function calls in 9.319 seconds

   Ordered by: internal time
   List reduced from 72 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   270121    1.317    0.000    1.317    0.000 {method 'reduce' of 'numpy.ufunc' objects}
    60000    1.168    0.000    2.240    0.000 _methods.py:77(_var)
    60000    0.979    0.000    1.503    0.000 {method 'choice' of 'mtrand.RandomState' objects}
    30000    0.823    0.000    3.498    0.000 _distn_infrastructure.py:1701(sf)
   660091    0.662    0.000    0.662    0.000 {numpy.core.multiarray.array}
    30000    0.393    0.000    7.388    0.000 stats.py:3271(ttest_ind)
    30000    0.383    0.000    1.418    0.000 _distn_infrastructure.py:549(argsreduce)
    60000    0.249    0.000    0.808    0.000 _methods.py:53(_mean)
       60    0.249    0.004    9.315    0.155 power.py:829(_compare_distributions)
   270000    0.197    0.000    0.473    0.000 numeric.py:464(asanyarray)
